In [1]:
import tensorflow as tf
import os
import pickle
import numpy as np

CIFAR_DIR = "./cifar-10-batches-py"
print(os.listdir(CIFAR_DIR))

['data_batch_1', 'readme.html', 'batches.meta', 'data_batch_2', 'data_batch_5', 'test_batch', 'data_batch_4', 'data_batch_3']


In [2]:
'''
可视化工具 tensorboard使用：
    1.制定面板图上显示的变量
    2.训练过程中将这些变量计算出来输出到文件中
    3.文件解析，./tensorboard --logdir=dir
'''


'''
图像增强有两种方法：
    1.在计算图的构建中运用到x_image上去
    2.在feed_dict之前进行图像增强
'''

'\n图像增强有两种方法：\n    1.在计算图的构建中运用到x_image上去\n    2.在feed_dict之前进行图像增强\n'

In [3]:
def load_data(filename):
    """read data from data file."""
    with open(filename, 'rb') as f:
        data = pickle.load(f, encoding='bytes')
        return data[b'data'], data[b'labels']

# tensorflow.Dataset.
class CifarData:
    def __init__(self, filenames, need_shuffle):
        all_data = []
        all_labels = []
        for filename in filenames:
            data, labels = load_data(filename)
            all_data.append(data)
            all_labels.append(labels)
        self._data = np.vstack(all_data)
        '''这里归一化会导致后面图像识别不出来，因此取消，转而在计算图中进行图像归一化'''
        self._data = self._data
        self._labels = np.hstack(all_labels)
        print(self._data.shape)
        print(self._labels.shape)
        
        self._num_examples = self._data.shape[0]
        self._need_shuffle = need_shuffle
        self._indicator = 0
        if self._need_shuffle:
            self._shuffle_data()
            
    def _shuffle_data(self):
        # [0,1,2,3,4,5] -> [5,3,2,4,0,1]
        #将0-self._num_example的数进行一个混合排列----数据及下标的混合排列
        p = np.random.permutation(self._num_examples)
        self._data = self._data[p]
        self._labels = self._labels[p]
    
    def next_batch(self, batch_size):
        """return batch_size examples as a batch."""
        end_indicator = self._indicator + batch_size
        if end_indicator > self._num_examples:
            if self._need_shuffle:
                self._shuffle_data()
                self._indicator = 0
                end_indicator = batch_size
            else:
                raise Exception("have no more examples")
        if end_indicator > self._num_examples:
            raise Exception("batch size is larger than all examples")
        batch_data = self._data[self._indicator: end_indicator]
        batch_labels = self._labels[self._indicator: end_indicator]
        self._indicator = end_indicator
        return batch_data, batch_labels

train_filenames = [os.path.join(CIFAR_DIR, 'data_batch_%d' % i) for i in range(1, 6)]
test_filenames = [os.path.join(CIFAR_DIR, 'test_batch')]

train_data = CifarData(train_filenames, True)
test_data = CifarData(test_filenames, False)

(50000, 3072)
(50000,)
(10000, 3072)
(10000,)


In [4]:
batch_size = 20
x = tf.placeholder(tf.float32, [batch_size, 3072])
y = tf.placeholder(tf.int64, [batch_size])

is_training = tf.placeholder(tf.bool,[])

# [None], eg: [0,5,6,3]
x_image = tf.reshape(x, [-1, 3, 32, 32])
# 32*32
x_image = tf.transpose(x_image, perm=[0, 2, 3, 1])
'''图像增强实现'''
#图像增强api都是基于3维的，不能在训练过程中能使用--split-concat模式
x_image_arr = tf.split(x_image, num_or_size_splits=batch_size, axis=0)
result_x_image_arr = []
for x_single_image in x_image_arr:
    #x_single_image--->[1,32,32,3]
    x_single_image = tf.reshape(x_single_image, [32,32,3])
    data_aug_1 = tf.image.random_flip_left_right(x_single_image)
    data_aug_2 = tf.image.random_brightness(data_aug_1, max_delta=63)#里面参数为改变光照强度的最大值
    data_aug_3 = tf.image.random_contrast(data_aug_2, lower=0.2, upper=1.8)#改变对比度的区间
    x_single_image = tf.reshape(data_aug_3, [1,32,32,3])
    result_x_image_arr.append(x_single_image)
result_x_images = tf.concat(result_x_image_arr, axis=0)


#归一化
normal_result_x_images = result_x_images / 127.5 - 1

'''批归一化使用和对比'''

'''
def conv_wraper(inputs, 
                        name, 
                        output_channel=32, 
                        kernel_size=(3,3),
                        padding='same',
                        activation=tf.nn.relu):

    return tf.layers.conv2d(inputs, 
                                     output_channel, 
                                     kernel_size, 
                                     padding='same', 
                                     activation=tf.nn.relu, 
                                     name=name)

'''

'''批归一化:
    outwith batch normalization:  conv --> activation
    with batch normalization: conv --> bn -->  activation'''
def conv_wraper(inputs, 
                        name, 
                        is_training,
                        output_channel=32, 
                        kernel_size=(3,3),
                        padding='same',
                        activation=tf.nn.relu):
    '''wrapper of tf.layers.conv2d'''
    with tf.name_scope(name):
        conv2d = tf.layers.conv2d(inputs, 
                                             output_channel, 
                                             kernel_size, 
                                             padding='same', 
                                             activation=None, 
                                             name= name+'conv1')
        
        bn = tf.layers.batch_normalization(conv2d,
                                                        training= is_training)#因为训练和测试模式上bn不一样
        
        return activation(bn)

def poo ling_wraper(inputs, name):
    '''wrapper of tf.maxpooling2d'''
    return tf.layers.max_pooling2d(inputs,
                                                (2,2),
                                                (2,2),
                                                name=name)


# conv1: 神经元图， feature_map, 输出图像
conv1_1 = conv_wraper(normal_result_x_images, 'conv1_1', is_training)
conv1_2 = conv_wraper(conv1_1, 'conv1_2', is_training)
conv1_3  = conv_wraper(conv1_2, 'conv1_3', is_training)
pooling1 = pooling_wraper(conv1_3,  'pooling1')

conv2_1 =  conv_wraper(pooling1, 'conv2_1', is_training)
conv2_2= conv_wraper(conv2_1,'conv2_2', is_training)
conv2_3 = conv_wraper(conv2_2, 'conv2_3', is_training)
pooling2 = pooling_wraper(conv2_3, 'pooling2')

conv3_1 = conv_wraper(pooling2, 'conv3_1', is_training)
conv3_2  = conv_wraper(conv3_1, 'conv3_2', is_training)
conv3_3 = conv_wraper(conv3_2,'conv3_3', is_training)
pooling3 = pooling_wraper(conv3_3,'pooling3')


'''------展平------'''
# [None, 4 * 4 * 32]
flatten = tf.layers.flatten(pooling3)
y_ = tf.layers.dense(flatten, 10)

loss = tf.losses.sparse_softmax_cross_entropy(labels=y, logits=y_)
# y_ -> sofmax
# y -> one_hot
# loss = ylogy_

# indices
predict = tf.argmax(y_, 1)
# [1,0,1,1,1,0,0,0]
correct_prediction = tf.equal(predict, y)
accuracy = tf.reduce_mean(tf.cast(correct_prediction, tf.float64))

with tf.name_scope('train_op'):
    train_op = tf.train.AdamOptimizer(1e-3).minimize(loss)

In [5]:
'''tensorboard中定义变量需要在网络定义完成之后再定义变量
tensorboard的所有api都在tf.summary下'''

def variable_summary(var,name):
    """ 给一个变量的很多统计量建立一个summary，可以输出到直方图中可以看到，这个函数可以运用到计算中的各个节点上去，tensorboard就能看到各个统计量了"""
    with tf.name_scope(name):
        mean = tf.reduce_mean(var)
        with tf.name_scope('stddev'):
            stddev = tf.sqrt(tf.reduce_mean(tf.square(var - mean)))
        tf.summary.scalar('mean',mean)
        tf.summary.scalar('stddev',stddev)
        tf.summary.scalar('min',tf.reduce_min(var))
        tf.summary.scalar('max',tf.reduce_max(var))
        '''打印直方图'''
        tf.summary.histogram('histogram',var)
         

'''将卷积层的输出分布放到功能函数中看看输出是什么'''
with tf.name_scope('summary'):
    variable_summary(conv1_1, 'conv1_1')
    variable_summary(conv1_2, 'conv1_2')
    variable_summary(pooling1, 'pooling1')
    variable_summary(conv2_1, 'conv2_1')
    variable_summary(conv2_2, 'conv2_2')
    variable_summary(pooling2, 'pooling2')
    variable_summary(conv3_1, 'conv3_1')
    variable_summary(conv3_2, 'conv3_2')
    variable_summary(pooling3, 'pooling3')
    variable_summary(y_, 'y_')
    '''注意，这些都没定义返回值，merge_all（）会自动手集所有调用过tf.summary.scala的节点，而merge则需要指定这就需要返回值了'''


    #'loss':<10,1.1>,<20,1.08>
loss_summary = tf.summary.scalar('loss', loss)

accuracy_summary = tf.summary.scalar('accuracy',accuracy)


inputs_summary = tf.summary.image('input_image', normal_result_x_images)

'''因为有很多变量，如果所有summary都进行sess。run（）的话不合适，太长，tensorflow提供了跟好的api '''
merged_summary = tf.summary.merge_all()#将上面所有summary聚合起来

#制定某些summary来做聚合
merged_summary_test = tf.summary.merge([loss_summary,accuracy_summary,inputs_summary])


'''在tensorboard中指定文件夹不需要自己定义文件名，库会自己做，我们只需要制定文件夹就行了'''
LOG_DIR = '.'
run_label = 'run_vgg_tensorboard'
run_dir = os.path.join(LOG_DIR,run_label)
if not os.path.exists(run_dir):
    os.mkdir(run_dir)#tensorboard不会自动创建文件夹自己写程序创建和判断

train_log_dir = os.path.join(run_dir,'train')
test_log_dir = os.path.join(run_dir,'test')
if not os.path.exists(train_log_dir):
    os.makedirs(train_log_dir)
    
if not os.path.exists(test_log_dir):
    os.makedirs(test_log_dir)
    

In [7]:
init = tf.global_variables_initializer()
batch_size = 20
train_steps = 1000
test_steps = 100
output_summary_every_steps = 100

# train 10k: 71.35%
with tf.Session() as sess:
    sess.run(init)
    
    '''tensorboard文件输出部分'''
    train_writer = tf.summary.FileWriter(train_log_dir,sess.graph)#写入指定文件夹，兵指定计算图
    test_write = tf.summary.FileWriter(test_log_dir)#此处并没有指定计算图
   
    #train中每次是调用了nextbatch，数据是无序的，在test中我们希望它是有序的
    fixed_test_batch_data, fixed_test_batch_labels = test_data.next_batch(batch_size)
    
    for i in range(train_steps):
        batch_data, batch_labels = train_data.next_batch(batch_size)
        '''tensorboard,想要将summary输出出来，首先要计算出来，加一个变量就行了'''
        eval_ops = [loss,accuracy,train_op]
        should_out_put_summary = ((i+1)%output_summary_every_steps == 0)
        if should_out_put_summary:
            eval_ops.append(merged_summary)
        eval_ops_results = sess.run(
            eval_ops,
            feed_dict={
                x: batch_data,
                y: batch_labels,
                is_training: True})
        loss_val, acc_val = eval_ops_results[0:2]
        if should_out_put_summary:
            train_summary_str = eval_ops_results[-1]
            train_writer.add_summary(train_summary_str,i+1) #指定步数，随着训练步数的值的变化而变化的点
            test_summary_str = sess.run([merged_summary_test],feed_dict={x: fixed_test_batch_data,
                                                                                                       y:fixed_test_batch_labels,
                                                                                                       is_training: False})[0]
            test_write.add_summary(test_summary_str,i+1)
            
   
            
        if (i+1) % 100 == 0:
            print('[Train] Step: %d, loss: %4.5f, acc: %4.5f' 
                  % (i+1, loss_val, acc_val))
        if (i+1) % 1000 == 0:
            test_data = CifarData(test_filenames, False)
            all_test_acc_val = []
            for j in range(test_steps):
                test_batch_data, test_batch_labels \
                    = test_data.next_batch(batch_size)
                test_acc_val = sess.run(
                    [accuracy],
                    feed_dict = {
                        x: test_batch_data, 
                        y: test_batch_labels,
                        is_training: False
                    })
                all_test_acc_val.append(test_acc_val)
            test_acc = np.mean(all_test_acc_val)
            print('[Test ] Step: %d, acc: %4.5f'
                  % (i+1, test_acc))

[Train] Step: 100, loss: 2.14216, acc: 0.25000
[Train] Step: 200, loss: 2.12369, acc: 0.25000
[Train] Step: 300, loss: 1.95284, acc: 0.35000
[Train] Step: 400, loss: 2.14379, acc: 0.20000
[Train] Step: 500, loss: 2.36095, acc: 0.15000
[Train] Step: 600, loss: 1.47636, acc: 0.50000
[Train] Step: 700, loss: 1.73339, acc: 0.15000
[Train] Step: 800, loss: 1.69389, acc: 0.40000
[Train] Step: 900, loss: 1.66204, acc: 0.30000
[Train] Step: 1000, loss: 1.52863, acc: 0.35000
(10000, 3072)
(10000,)
[Test ] Step: 1000, acc: 0.17500


### tensorboard打开方式
    tensorboard --logdir=文件夹 名（单个）
     tensorboard --logdir=文件夹1：'自己单独命名1'，文件夹2:‘自己命名2’